# Model_V0


In [1]:
from pathlib import Path
import os

import numpy as np
import matplotlib.pyplot as plt

from skimage.color import rgb2lab, rgb2gray, lab2rgb
from skimage.io import imread, imshow, imsave

from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, UpSampling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

---
## Create `ImageDataGenerators` for train and validation sets

In [14]:
# Create image generator for image augmentation during training
def TrainImage_a_b_gen(Xtrain, batch_size, validation_split):
    '''Function that gets the training images using the flow method
    via train_generator and applies the data augmentation of train_datagen.
    This function is then used in the .fit() method when training the model.
    
    Note that this returns a tuple of (inputs, targets),
    where inputs is a np.array with shape (batch_size, h, w, 1)
    and targets is a np.array with shape (batch_sze, h, w, 2).'''
    
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                       shear_range = 0.4,
                                       zoom_range = 0.4,
                                       rotation_range = 45,
                                       horizontal_flip = True,
                                       validation_split = validation_split)
    
    train_generator = train_datagen.flow(x = Xtrain,
                                         batch_size = batch_size,
                                         subset = "training",
                                         save_to_dir = "Images/PokemonSilver_AugmentedTrain", 
                                         save_prefix = "PokemonSilver_Augmented",
                                         save_format = "png",
                                         shuffle = True,
                                         seed = 42)
    
    # Generate the tuple (X, Y) for each training batch
    for batch in train_generator:
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        Y_batch = lab_batch[:, :, :, 1:]
        Y_batch /= 128 # Normalise the values to [-1.0, 1.0]
        yield (X_batch.reshape(X_batch.shape + (1, )), Y_batch)
        

def ValidationImage_a_b_gen(Xtrain, batch_size, validation_split):
    
    validation_datagen = ImageDataGenerator(rescale = 1./255,
                                            validation_split = validation_split)
    
    validation_generator = validation_datagen.flow(x = Xtrain,
                                                   batch_size = batch_size,
                                                   subset = "validation",
                                                   save_to_dir = "Images/PokemonSilver_Validation", 
                                                   save_prefix = "PokemonSilver_Validation",
                                                   save_format = "png",
                                                   shuffle = True,
                                                   seed = 42)
    
    # Generate the tuple (X, Y) for each validation batch
    for batch in validation_generator:
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        Y_batch = lab_batch[:, :, :, 1:]
        Y_batch /= 128
        yield (X_batch.reshape(X_batch.shape + (1, )), Y_batch)
        

---
## Model architecture

In [3]:
model = Sequential(name = "Colouring-Pokemon")

model.add(InputLayer(input_shape = (288, 320, 1)))
model.add(Conv2D(name = "Conv2D_8a_3x3", filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(Conv2D(name = "Conv2D_8b_3x3", filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_16a_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_16b_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(Conv2D(name = "Conv2D_32a_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_32b_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(UpSampling2D(name = "UpSamp-a_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_32c_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D(name = "UpSamp-b_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_16c_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D(name = "UpSamp-c_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_2_3x3", filters = 2, kernel_size = (3, 3), activation = 'tanh', padding = 'same'))

In [4]:
model.compile(optimizer = 'rmsprop', loss = 'mse')

print(model.summary())

Model: "Colouring-Pokemon"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_8a_3x3 (Conv2D)       (None, 144, 160, 8)       80        
_________________________________________________________________
Conv2D_8b_3x3 (Conv2D)       (None, 144, 160, 8)       584       
_________________________________________________________________
Conv2D_16a_3x3 (Conv2D)      (None, 144, 160, 16)      1168      
_________________________________________________________________
Conv2D_16b_3x3 (Conv2D)      (None, 72, 80, 16)        2320      
_________________________________________________________________
Conv2D_32a_3x3 (Conv2D)      (None, 72, 80, 32)        4640      
_________________________________________________________________
Conv2D_32b_3x3 (Conv2D)      (None, 36, 40, 32)        9248      
_________________________________________________________________
UpSamp-a_2x2 (UpSampling2D)  (None, 72, 80, 32)  

---
## Get train images from folder

In [5]:
# Get train images from folder
Xtrain = []
Xtrain_dir = Path("Images/PokemonSilver")

# Load image as numpy arrays to the list Xtrain
for filename in os.listdir(Xtrain_dir):
    if not filename.startswith('.'):
        img_dir = Xtrain_dir / filename
        Xtrain.append(img_to_array(load_img(img_dir)))

# Convert the list of arrays into a 4D numpy array
Xtrain = np.array(Xtrain, dtype = float)

In [6]:
print("Xtrain shape:", Xtrain.shape, type(Xtrain))

Xtrain shape: (6433, 288, 320, 3) <class 'numpy.ndarray'>


---
## Train the model

https://lambdalabs.com/blog/tensorflow-2-0-tutorial-03-saving-checkpoints/

Training logs:

**training_run01**
 - Size of training set = approx. 2000
 - Epochs = 10
 - Batch size = 20
 - Steps per epoch = 50
 - Validation steps = 1
 
**training_run02**
 - Size of training set = approx. 6000
 - Epochs = 50
 - Batch size = 20
 - Steps per epoch = 289
 - Validation steps = 1

In [7]:
def train_and_save_model(training_run, Xtrain, model, validation_split, batch_size, epochs, steps_per_epoch, validation_steps, checkpoint_filepath):
    
    '''Function to train the model and save using ModelCheckpoint and EarlyStopping.'''
    
    print("=============================")
    print(f" ##### TRAINING RUN {training_run} #####")
    print("=============================")
    print()
    print()
    
    # Print model summary
    print("===============")
    print(" MODEL SUMMARY ")
    print("===============")
    print(model.summary())
    print()
    print()
    
    # Print information about training and validation set
    print("===========================")
    print(" TRAIN/VAL DATASET SUMMARY ")
    print("===========================")
    total_train_size = len(Xtrain)
    train_size = round((1 - validation_split) * total_train_size)
    validation_size = round(validation_split * total_train_size)
    print(f"Size of total training set    : {len(Xtrain)}")
    print(f"(Training, validation) split  : ({1 - validation_split}, {validation_split})")
    print(f"Size of training set          : {1 - validation_split} * {total_train_size} = {train_size}")
    print(f"Size of validation set        : {validation_split} * {total_train_size} = {validation_size}")
    print()
    print()
    
    # Checkpoint
    checkpoint = ModelCheckpoint(filepath = checkpoint_filepath,
                                 monitor = "val_loss",
                                 mode = "min",
                                 save_best_only = True,
                                 save_weights_only = False,
                                 save_freq = "epoch",
                                 verbose = 1)
    
    # Early Stop
    earlystop = EarlyStopping(monitor = "val_loss",
                              mode = "min",
                              patience = 10,
                              restore_best_weights = True,
                              verbose = 1)
    
    callbacks_list = [checkpoint, earlystop]
    
    # Train model
    print("=====================================")
    print(" TRAIN, VALIDATE AND SAVE CHECKPOINT")
    print("=====================================")
    model_info = model.fit(x = TrainImage_a_b_gen(Xtrain, batch_size, validation_split),
                           epochs = epochs,
                           steps_per_epoch = steps_per_epoch,
                           validation_data = ValidationImage_a_b_gen(Xtrain, batch_size, validation_split),
                           validation_steps = validation_steps,
                           callbacks = callbacks_list,
                           verbose = 2)
    
    print()
    print()
    print("##### END OF TRAINING RUN. #####")

In [13]:
train_and_save_model(training_run = "02", 
                     Xtrain = Xtrain,
                     model = model,
                     validation_split = 0.1,
                     batch_size = 10,
                     epochs = 50,
                     steps_per_epoch = 1,
                     validation_steps = 1,
                     checkpoint_filepath = "SavedCheckpoints/trainingrun02/model.{epoch:02d}-{val_loss:.6f}.hdf5")



 ##### TRAINING RUN 02 #####


 MODEL SUMMARY 
Model: "Colouring-Pokemon"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_8a_3x3 (Conv2D)       (None, 144, 160, 8)       80        
_________________________________________________________________
Conv2D_8b_3x3 (Conv2D)       (None, 144, 160, 8)       584       
_________________________________________________________________
Conv2D_16a_3x3 (Conv2D)      (None, 144, 160, 16)      1168      
_________________________________________________________________
Conv2D_16b_3x3 (Conv2D)      (None, 72, 80, 16)        2320      
_________________________________________________________________
Conv2D_32a_3x3 (Conv2D)      (None, 72, 80, 32)        4640      
_________________________________________________________________
Conv2D_32b_3x3 (Conv2D)      (None, 36, 40, 32)        9248      
_________________________________________________________________
Up

---
## Get test images from folder

In [9]:
colourme = []
colourme_dir = Path("Images/PokemonBlue")

# Load image as numpy arrays to the list X
for filename in os.listdir(colourme_dir):
    if not filename.startswith('.'):
        img_dir = colourme_dir / filename
        colourme.append(img_to_array(load_img(img_dir)))

colourme = np.array(colourme, dtype = float)
colourme = rgb2lab(1.0/255 * colourme)[:, :, :, 0]
colourme = colourme.reshape(colourme.shape + (1, ))

In [10]:
print("colourme shape:", colourme.shape, type(colourme))

colourme shape: (826, 288, 320, 1) <class 'numpy.ndarray'>


---
## Test the model (i.e. use for prediction)

In [11]:
# Test the model on colour_me
output = model.predict(colourme)
output *= 128

---
## Save colourised images

In [12]:
# Output colourisations of colour_me
for i in range(len(output)):
    output_array = np.zeros((288, 320, 3))
    output_array[:, :, 0] = colourme[i][:, :, 0]
    output_array[:, :, 1:] = output[i]
    output_array = lab2rgb(output_array)
    output_array = (output_array * 255).astype(np.uint8) # Convert to uint8 to avoid lossy conversion warning
    imsave(f"Outputs_02/PokemonBlue_predicted{i}.png", output_array)

/Users/cleonwong/Desktop/Code/python-virtual-environments/Colouring-Pokemon_env/lib/python3.7/site-packages/skimage/color/colorconv.py:1128: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/Users/cleonwong/Desktop/Code/python-virtual-environments/Colouring-Pokemon_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Outputs_02/PokemonBlue_predicted49.png is a low contrast image
  
/Users/cleonwong/Desktop/Code/python-virtual-environments/Colouring-Pokemon_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Outputs_02/PokemonBlue_predicted66.png is a low contrast image
  
/Users/cleonwong/Desktop/Code/python-virtual-environments/Colouring-Pokemon_env/lib/python3.7/site-packages/skimage/color/colorconv.py:1128: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/Users/cleonwong/Desktop/Code/python-virtual-environments/Colouring-Pokemon_

KeyboardInterrupt: 